### What does this script do?
The following is a Python script which cleans and normalizes metadata in accordance with the Open Index Map (OIM) specification: https://openindexmaps.org/specification/1.0.0. These metadata come from a set of manually-created map-inventory spreadsheets, which are combined with digital map indexes to produce GeoJSON files. (See more at https://github.com/ubc-lib-geo/creating-index-maps.)

This script:
- Imports all GeoJSON files within each continent/region subdirectory in the 'spatial-indexes' repository of the 'ubc-lib-geo' GitHub organization: https://github.com/ubc-lib-geo/spatial-indexes
- Combines all files into a single object (a GeoPandas GeoDataFrame)
- Compares the column labels (element titles) against the OIM specification
    - These element titles were input into a spreadsheet and then imported into this script
    - For instances where the element title does not match the OIM specification *and* there exists a corresponding field to map the data to (e.g., 'notes' and 'note' fields)--meaning the relevant data is split between more than one column--data is transferred from the non-matching field to the OIM-compliant field
        - After completing this, all non-compliant fields are removed
    - For instances where the element title does not match the OIM specification, *but* all relevant data is located in that single column, the column label is changed to match the element title in the OIM specification
    - For instances where the element title not only does not match the OIM specification, *but* no corresponding element title exists within the specification (e.g., 'editionNotes'), data will be mapped to another selected field (e.g., 'edition' or 'note')
        - If data already exists in the selected field (e.g., 'edition' or 'note'), the data from the no-equivalent field (e.g., 'editionNotes') will be appended to it using the separator '; '
- Removes any extra white space at the beginning and end of all values
- For fields with an OIM-specified data type of boolean (i.e., 'available', 'contLines', 'bathLines'), existing values are mapped to the form 'true' or 'false', as outlined in the OIM specification
    - Files containing null values in these fields are identified
    - With the exception of the 'available' field, null values are left as null values (i.e., they are not converted to either 'true' or 'false')
- The combined data is separated into individual GeoJSONs and written-out as individual files to their respective continent/region subdirectory in the 'spatial-indexes' repository
    - Files' original names are maintained
    - Fields containing only null values within an individual file are removed before being written-out

In [1]:
# Import required libraries
import os # For selecting files within a directory
import pandas as pd # For data cleaning and manipulation
import geopandas as gpd # For working with spatial data and GeoJSON files

In [3]:
# Select only the GeoJSON files within the 'spatial-indexes' GitHub repository
# These files are stored in multiple subdirectories by continent/world region

# List the names of the continents/regions subdirectories in the 'spatial-indexes' repository
# Uses fact that folders lack file extensions to select subdirectories
continents_regions = [item for item in os.listdir() if '.' not in item]

geodataframes = []

for directory in continents_regions:
    os.chdir(directory)
    for file in directory:
        #if file.endswith('.geojson'):
        geodataframe = gpd.read_file(file)
        geodataframe['fileName'] = file
        geodataframe['directoryName'] = directory
        geodataframes.append(geodataframe)
    os.chdir('..')

all_geojsons = pd.concat(geodataframes)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "fiona\ogrext.pyx", line 136, in fiona.ogrext.gdal_open_vector
  File "fiona\_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: a: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\badimond\Desktop\maps_project\spatial-indexes\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\badimond\AppData\Local\Temp\ipykernel_14268\3496819265.py", line 14, in <module>
    geodataframe = gpd.read_file(file)
  File "C:\Users\badimond\Desktop\maps_project\spatial-indexes\.venv\lib\site-packages\geopandas\io\file.py", line 259, in _read_file
    return _read_file_fiona(
  File "C:\Users\badimond\Desktop\maps_project\spatial-indexes\.venv\lib\site-packages\geopandas\io\file.py", line 303, in _read_file_fiona
    with reader(path_or

### Element titles from OIM specification

In [6]:
# Import and list (in alphabetical order) all element titles from the OIM specification
elements_from_template = pd.Series(pd.read_csv('attribute_table_template_OIM_v1_0_0.csv').columns).sort_values()
elements_from_template

27     available
19    bathInterv
18     bathLines
24         color
17    contInterv
16     contLines
6           date
8      datePhoto
5        datePub
9     dateReprnt
7     dateSurvey
29       digHold
32      download
11       edition
36      fileName
21      geometry
35       iiifUrl
25          inst
30    instCallNo
0          label
1       labelAlt
2      labelAlt2
15      lcCallNo
22      location
37          note
13      overlays
10     overprint
28      physHold
20      primeMer
14    projection
12     publisher
31         recId
23         scale
26       sheetId
34      thumbUrl
3          title
4       titleAlt
33    websiteUrl
dtype: object

### Element titles from GeoJSON files

In [7]:
# Retrieve and list (in alphabetical order) all element titles from the selected GeoJSON files
elements_from_geojsons = pd.Series(all_geojsons.columns).sort_values()
elements_from_geojsons

22        available
42        avaliable
19          bathInt
18        bathLines
38            color
17          contInt
43       contInterv
16        contLines
6              date
4         datePhoto
2           datePub
5       dateReprint
3        dateSurvey
24          digHold
41       digHolding
33    directoryName
27         download
46             east
11          edition
12     editionNotes
32         fileName
31         geometry
36          iiifURL
29          iiifUrl
20             inst
35         instCall
25       instCallNo
0             label
1          labelAlt
34           lcCall
15         lcCallNo
7          location
45            north
40             note
30            notes
23         physHold
14       projection
13        publisher
26            recId
8             scale
39          sheetID
21          sheetId
47            south
9             title
10         titleAlt
28       websiteUrl
44             west
37             year
dtype: object

### Element titles in GeoJSON files that do not match OIM specification

In [8]:
# Check (and return) if any elements from the GeoJSON files do no match elements from the OIM specification
# This cell returns any elements that do not match and whose titles need to be changed

spec_check_mask = elements_from_geojsons.isin(elements_from_template)
non_matching_elements = elements_from_geojsons[~spec_check_mask]
non_matching_elements

42        avaliable
19          bathInt
17          contInt
5       dateReprint
41       digHolding
33    directoryName
46             east
12     editionNotes
36          iiifURL
35         instCall
34           lcCall
45            north
30            notes
39          sheetID
47            south
44             west
37             year
dtype: object

### Classify element titles
Classify the incorrect element titles above according to these three scenarios:
1. The correct element title *is* present in our combined dataset of GeoJSON files
    - **User action:**
        - Compare the list above to the element titles from the GeoJSON files
    - **Script actions:**
        - Data will be transferred from the field with the incorrect title to the field with the correct title
        - Fields with incorrect titles (which are now empty) will be dropped
2. The correct element title *is not* present in our combined dataset of GeoJSON files
    - **User action:**
        - Compare the list above to the element titles from the GeoJSON files
    - **Script action:**
        - Since all the relevant data is contained in a single field (not split across multiple fields), the field will simply be renamed to the correct title
3. The incorrect element title has no corresponding OIM equivalent (e.g., 'editionNotes')
    - **User actions:**
        - Compare the list above to the element titles from the OIM specification
        - If you identify a non-OIM-equivalent title, decide which field to transfer these data to (e.g., 'edition' or 'note')
    - **Script actions:**
        - Determine whether the field(s) to transfer these data to already contains data
        - If so, the data from the incorrect field(s) will be appended to the data in the correct field
        - Fields with no equivalent OIM title (which are now empty) will be dropped

### Scenario 1

#### USER INPUT CELL

In [9]:
# Data in fields with incorrect OIM titles must be transferred to the corresponding field with the
# correct title

# Review 'elements_from_geojsons' list, and identify all instances where there are multiple variants
# of a single element title (e.g., 'notes' and 'note')

# Identify and add these in the 'mappings' list below
# Incorrect element titles are on the left; their corresponding correct element title is on the right
# NOTE: There may be multiple incorrect values on the left that map to the same correct value on the
# right

mappings = [
    ('notes', 'note'),
    ('lcCall', 'lcCallNo'),
    ('instCall', 'instCallNo'),
    ('iiifURL', 'iiifUrl'),
    ('year', 'date'),
    ('sheetID', 'sheetId'),
    ('digHolding', 'digHold'),
    ('avaliable', 'available'),
    ('contInt', 'contInterv')
]

for i in mappings:
    if i[0] in elements_from_geojsons:
        all_geojsons.loc[all_geojsons[i[0]].notnull(), i[1]] = all_geojsons.loc[all_geojsons[i[0]].notnull(), i[0]]
    else:
        print(i[0] + ' is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.')

notes is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
lcCall is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
instCall is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
iiifURL is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
year is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
sheetID is not present in the elements_from_geojsons list. Please ensure the spelling of this title matches how it appears in the elements_from_geojsons list.
digHolding is not present in the elements_from_geoj

In [10]:
# Now that the data from fields with incorrect element titles have been transferred to fields with
# correct element titles, the fields with incorrect element titles can be dropped from the
# GeoDataFrame

for i in mappings:
    if i[0] in elements_from_geojsons:
        duplicate_fields = [i[0] for i in mappings]
        all_geojsons = all_geojsons.drop(columns = duplicate_fields)
    else:
        print(i[0] + ' is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.')

notes is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.
lcCall is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.
instCall is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.
iiifURL is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.
year is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell matches how it appears in the elements_from_geojsons list.
sheetID is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the above cell ma

### Scenario 2

#### USER INPUT CELL

In [11]:
# Rename those incorrect titles whose corresponding correct title does not exist within the GeoJSON
# files
# Incorrect names are on the left; correct names are on the right

all_geojsons = all_geojsons.rename(
    columns = {
        'bathInt': 'bathInterv',
        'dateReprint': 'dateReprnt'
    }
)

### Scenario 3

#### USER INPUT CELL

In [12]:
# List the incorrect titles that have no equivalent within the OIM specification on the left and the designated fields to transfer the data to on the right

no_equivalent = [
    ('editionNotes', 'note')
]

In [13]:
# Set all data types as 'string', with the exception of the 'geometry' field
# This will make appending values between fields easier
all_geojsons.loc[:, all_geojsons.select_dtypes('object').columns] = all_geojsons.select_dtypes('object').astype('string')

In [14]:
# For non-equivalent titles, transfer data to designated field
# In cases where designated field already contains data, append data from non-equivalent title to this field using '; ' as separator

if len(no_equivalent) != 0:
    for i in no_equivalent:
        if i[0] in elements_from_geojsons:
            for x in range(len(all_geojsons)):
                if (all_geojsons[i[1]].isna().iloc[x]) & (~all_geojsons[i[0]].isna().iloc[x]):
                    all_geojsons[i[1]].iloc[x] = all_geojsons[i[0]].iloc[x]
                elif (~all_geojsons[i[1]].isna().iloc[x]) & (all_geojsons[i[0]].isna().iloc[x]):
                    all_geojsons[i[1]].iloc[x] = all_geojsons[i[1]].iloc[x]
                else:
                    all_geojsons[i[1]].iloc[x] = all_geojsons[i[1]].iloc[x] + '; ' + all_geojsons[i[0]].iloc[x]
        else:
            print(i[0] + ' is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the mappings cell matches how it appears in the elements_from_geojsons list.')
else:
    print('You have determined that all element titles in the selected GeoJSON files have a corresponding title in the OIM specification.')

editionNotes is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the mappings cell matches how it appears in the elements_from_geojsons list.


In [15]:
# Now that the data from any field with no equivalent value in the OIM specification have been mapped
# to other selected fields, these fields can be dropped

if len(no_equivalent) != 0:
    for i in no_equivalent:
        if i[0] in elements_from_geojsons:
            no_corresponding_field = [i[0] for i in no_equivalent]
            all_geojsons = all_geojsons.drop(columns = no_corresponding_field)
        else:
            print(i[0] + ' is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the mappings cell matches how it appears in the elements_from_geojsons list.')
else:
    print('You have determined that all element titles in the selected GeoJSON files have a corresponding title in the OIM specification.')

editionNotes is not present in the elements_from_geojsons list. Please ensure the spelling of this title in the mappings cell matches how it appears in the elements_from_geojsons list.


## Data cleaning

### Remove extra white space from fields

In [16]:
# Create a list of 'string' type fields in the GeoDataFrame 'all_geojsons'
string_fields = list(all_geojsons.select_dtypes(include = 'object').columns)

In [17]:
# Remove extra white space
for field in string_fields:
    all_geojsons[field] = all_geojsons[field].str.strip()

### Check values in boolean fields
- These values must equal either 'true', 'false', or null
- Since Python booleans are of the form 'True'/'False', transforming these values must be done as strings, not booleans

### 'available' field

In [18]:
# Show the unique values for the field
list(all_geojsons['available'].unique())

['true',
 'false',
 'True',
 'False',
 'TRUE',
 'FALSE',
 'y',
 'n',
 <NA>,
 'yes',
 'no']

#### USER INPUT CELL

In [19]:
# Create and apply a mapping to transform the incorrect values to the correct values
# Incorrect values are on the left; correct values are on the right
# Null values (<NA>) can be left as null

available_mapping = {
    'TRUE': 'true',
    'True': 'true',
    'y': 'true',
    'FALSE': 'false',
    'False': 'false',
    'n': 'false'
}

all_geojsons['available'] = all_geojsons['available'].replace(available_mapping)

In [20]:
# Identify those files with null values in the 'available' field
list(all_geojsons.loc[all_geojsons['available'].isna(), 'fileName'].unique())

['canada_britishColumbia_columbiaRiverBasin_1958',
 'canada_britishColumbia_gvrd',
 'canada_britishColumbia_ifcsm']

In [21]:
# Replace null values with 'true'
# Assumes that inventory records are created for maps that are physically present and, therefore,
# available
all_geojsons.loc[all_geojsons['available'].isna(), 'available'] = 'true'

### 'contLines'

In [22]:
# Show the unique values for the field
list(all_geojsons['contLines'].unique())

['true', 'false', <NA>, 'no', 'TRUE']

#### USER INPUT CELL

In [23]:
# Create and apply a mapping to transform the incorrect values to the correct values
# Incorrect values are on the left; correct values are on the right
# Null values (<NA>) can be left as null

contLines_mapping = {
    'TRUE': 'true',
    'no': 'false'
}

all_geojsons['contLines'] = all_geojsons['contLines'].replace(contLines_mapping)

In [24]:
# Identify those files with null values in the 'contLines' field
list(all_geojsons.loc[all_geojsons['contLines'].isna(), 'fileName'].unique())

['china_250k_L532',
 'china_250k_L542',
 'china_250k_L581',
 'china_250k_L582',
 'indonesia_250k',
 'france_250k',
 'france_500k',
 'iceland_250k',
 'canada_250k_nts',
 'canada_25k_nts',
 'canada_500k_nts',
 'canada_50k_nts',
 'canada_alberta_50k_nts',
 'canada_britishColumbia_50k_nts',
 'canada_britishColumbia_columbiaRiverBasin_1958',
 'canada_britishColumbia_gvrd',
 'canada_manitoba_50k_nts',
 'canada_northwestTerritory_50k_nts',
 'canada_saskatchewan_50k_nts',
 'canada_yukonTerritory_50k_nts']

In [25]:
# # Replace null values with 'false'
# # Assumes any map without a value in this field does not contain contour lines
# all_geojsons.loc[all_geojsons['contLines'].isna(), 'contLines'] = 'false'

### 'bathLines'

In [26]:
# Show the unique values for the field
# Null values (<NA>) can be left as null
list(all_geojsons['bathLines'].unique())

['true', 'false', <NA>, 'FALSE']

#### USER INPUT CELL

In [27]:
# # Create and apply a mapping to transform the incorrect values to the correct values
# # Incorrect values are on the left; correct values are on the right
# # Null values (<NA>) can be left as null

# bathLines_mapping = {
#     'incorrect_a': 'correct_a',
#     'incorrect_b': 'correct_b'
# }

# all_geojsons['bathLines'] = all_geojsons['bathLines'].replace(bathLines_mapping)

In [28]:
# Identify those files with null values in the 'bathLines' field
list(all_geojsons.loc[all_geojsons['bathLines'].isna(), 'fileName'].unique())

['china_250k_L532',
 'china_250k_L542',
 'china_250k_L581',
 'china_250k_L582',
 'indonesia_250k',
 'france_250k',
 'france_500k',
 'iceland_250k',
 'canada_250k_nts',
 'canada_25k_nts',
 'canada_500k_nts',
 'canada_50k_nts',
 'canada_alberta_50k_nts',
 'canada_britishColumbia_50k_nts',
 'canada_britishColumbia_columbiaRiverBasin_1958',
 'canada_britishColumbia_gvrd',
 'canada_britishColumbia_ifcsm',
 'canada_britishColumbia_ubc',
 'canada_britishColumbia_yaletown',
 'canada_manitoba_50k_nts',
 'canada_northwestTerritory_50k_nts',
 'canada_saskatchewan_50k_nts',
 'canada_yukonTerritory_50k_nts']

In [29]:
# # Replace null values with 'false'
# # Assumes any map without a value in this field does not contain bathymetry lines
# all_geojsons.loc[all_geojsons['bathLines'].isna(), 'bathLines'] = 'false'

## Write files

In [31]:
# Drop 'fileName' field, remove fields with only null values, and write as individual GeoJSON files back to original continent/world region directory

for dictionary in directories_and_files:
    os.chdir(list(dictionary.keys())[0])
    for file in os.listdir():
        if file.endswith('.geojson'):
            gdf = all_geojsons.loc[all_geojsons['fileName'] == file.replace('.geojson', '')]
            gdf = gdf.drop(columns = ['directoryName', 'fileName'])
            for column in gdf.columns:
                if gdf[column].isna().sum() == len(gdf[column]):
                    gdf = gdf.drop(columns = column)
            gdf.to_file(file, driver = 'GeoJSON')
    os.chdir('..')

c:\Users\badimond\Anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\badimond\Anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\badimond\Anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\badimond\Anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\badimond\Anaconda3\lib\